In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Concatenate, Add
from tensorflow.keras.models import Model

def dilated_residual_inception_block(input_tensor, filters):
    branches = []
    dilation_rates = [2, 4, 8]

    branch = Conv1D(filters=filters, kernel_size=1, padding='same')(input_tensor)
    branch = BatchNormalization()(branch)
    branches.append(branch)

    for dilation_rate in dilation_rates:
        branch = Conv1D(filters=filters, kernel_size=1, padding='same')(branch)
        branch = BatchNormalization()(branch)
        branch = Activation('relu')(branch)
        
        branch = Conv1D(filters=filters, kernel_size=3, dilation_rate=dilation_rate, padding='same')(branch)
        branch = BatchNormalization()(branch)
        branch = Activation('relu')(branch)

        branches.append(branch)

    concatenated = Concatenate()(branches)
    residual_output = Add()([input_tensor, concatenated])

    return residual_output


def encoder_block(x, filters):
    # Encoder uses 1x4 Conv with stride 4
    x = Conv1D(filters, 4, strides=4, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def decoder_block(x, shortcut, filters):
    # Decoder uses 1x4 transposed Conv with stride 1 (since upsampling is separate)
    x = Conv1D(filters, 4, padding='same')(x)
    x = UpSampling1D(4)(x)
    x = Concatenate()([x, shortcut])  # skip connection from the encoder
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def build_respnet(input_shape):
    inputs = Input(shape=input_shape)
    features = inputs.shape[1]

    lvls = 8

    # Encoder
    e1 = encoder_block(inputs, 32)
    print(e1.shape)
    e2 = encoder_block(e1, 64)
    print(e2.shape)
    e3 = encoder_block(e2, 128)
    print(e3.shape)
    e4 = encoder_block(e3, 256)
    print(e4.shape)
    e5 = encoder_block(e4, 512)
    print(e5.shape)

    # Dilated residual inception block
    bridge = dilated_residual_inception_block(e5, 512)

    # Decoder
    d1 = decoder_block(bridge, e4, 256)
    d2 = decoder_block(d1, e3, 128)
    d3 = decoder_block(d2, e2, 64)
    d4 = decoder_block(d3, e1, 32)

    # Output layer
    outputs = Conv1D(1, 1, activation='sigmoid', padding='same')(d4)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Assuming input PPG signal is of shape (batch_size, 2048, 1)
input_shape = (2048, 1)
model = build_respnet(input_shape)
model.summary()

(None, 512, 32)
(None, 128, 64)
(None, 32, 128)
(None, 8, 256)
(None, 2, 512)


ValueError: Inputs have incompatible shapes. Received shapes (2, 512) and (2, 2048)

```
The proposed network is adapted from the IncResU-Net
network [10] which was made for 2D medical image segmentation
application. The architecture of the proposed fully
convolutional network for performing Respiration signal
extraction is shown in Fig. 1. The encoder section is divided
into eight levels to perform the downsampling operation. A
1D convolution operation of size 1 4 is used to downsample
the input features. Instead of carrying out the downsampling
operation using max-pooling, strided convolution is used
instead to improve training efficiency [11]. The downsampling
operation decreases the input size while increasing the
number of filters at each encoder by a factor of two till the
number of encoder filters are 512 after which subsequent
encoder levels are maintained at 512 filters. In each encoder
level, 1D Convolution with stride 4 is applied followed by
Batch Normalization and leaky ReLU (with slope 0.2).
The output of each encoder level is then provided to the
dilated residual inception block is seen in Fig. 2. Usage
of the dilated residual inception block provides a larger
receptive field without a significant increase in parameters.
Further the use of residual connections within the block
is meant to greatly reduce the vanishing gradient problem
and reduce the convergence time during training [12]. The
decoder section of the proposed network utilizes feature
concatenation between the feature map of the decoder block
along with its corresponding encoder pair at the respective
level similar to the original U-Net [6]. After performing
convolution and dilated residual convolution operation, upsampling
is performed using a deconvolution operation at
each level of the decoder. In the final level of the decoder
1 1 1D convolution operation is performed to map the
features channels to the desired number of output channels.

```

In [3]:
from tensorflow.keras.layers import LeakyReLU
def dilated_residual_inception_block(input_tensor, filters):
    branches = []
    dilation_rates = [2, 4, 8]

    branch = Conv1D(filters=filters, kernel_size=1, padding='same')(input_tensor)
    branch = BatchNormalization()(branch)
    branches.append(branch)

    for dilation_rate in dilation_rates:
        branch = Conv1D(filters=filters, kernel_size=1, padding='same')(branch)
        branch = BatchNormalization()(branch)
        branch = Activation('relu')(branch)
        
        branch = Conv1D(filters=filters, kernel_size=3, dilation_rate=dilation_rate, padding='same')(branch)
        branch = BatchNormalization()(branch)
        branch = Activation('relu')(branch)

        branches.append(branch)

    concatenated = Concatenate()(branches)
    residual_output = Add()([input_tensor, concatenated])

    return residual_output

def encoder_block(x, filters):
    # Encoder uses 1x4 Conv with stride 4
    x = Conv1D(filters, 4, strides=4, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x
    
x = Input(shape=(2048, 1))

features = x.shape[2]

lvls = 8

for i in range(lvls):
    features = features*2
    print(f"Lvl: {i+1}")
    print(f"input shape: {x.shape}")
    print(f"features: {features}")
    x = dilated_residual_inception_block(x, features)
    print(f"after diluted conv shape: {x.shape}")
    x = encoder_block(x, features)
    print(f"after encoder downsampling shape: {x.shape}")

Lvl: 1
input shape: (None, 2048, 1)
features: 2
after diluted conv shape: (None, 2048, 8)
after encoder downsampling shape: (None, 2048, 2)
Lvl: 2
input shape: (None, 2048, 2)
features: 4


ValueError: Inputs have incompatible shapes. Received shapes (2048, 2) and (2048, 16)